# <p style="text-align: center;">**Bootcamp Big Data & Machine Learning V</p>**

**<p style="text-align: center;">NLP - Práctica Final</p>**</br>
**<p style="text-align: center;">Alberto Muñoz Freán</p>**

**3) NLG:**

En tercer lugar, implementaremos un modelo de generación de lenguaje que intentará imitar el estilo literario y el contenido de los [tweets](https://www.kaggle.com/speckledpingu/RawTwitterFeeds?select=NeildeGrasseTysonTweets.csv) de Neil deGrasse Tyson:

In [1]:
#Importamos todo lo necesario para hacer NLG:
import numpy as np
import pandas as pd
from copy import deepcopy

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

import re
import string

pd.options.display.max_colwidth = 180

Using TensorFlow backend.
C:\Users\amfre\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\amfre\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\amfre\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\amfre\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
#Cargamos los datos del csv con los tweets para entrenar el modelo:
data = pd.read_csv('Neil.csv')

#Nos quedamos únicamente con el campo "text" y convertimos ese dataframe en una lista:
data2 = data['text'].to_string()

#Al tratarse de tweets, hay elementos no deseados, como hashtags, menciones o hyperlinks. 
#Eliminaremos estos tres elementos mediante regex, en la medida de lo posible: 

#Hyperlinks:
data3 = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', ' ', data2, flags=re.MULTILINE)
data3 = re.sub(r'[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', ' ', data3, flags=re.MULTILINE)

#Menciones y hashtags:
data3 = ''.join(re.sub('(@[A-Za-z0–9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)',' ',data3))

#Múltiples espacios y caracteres residuales:
data3 = ''.join(re.sub(' \d+', ' ', data3)) 
data3 = " ".join(data3.split())
data3 = data3.replace('0 ', '')

#Comprobamos el resultado. Es evidente que no es perfecto, pero también se puede observar que ha eliminado bastantes instancias:
data3

'Moon s shadow landfalls Oregon crosses USA at mph exits SCarolina Behold Muuurica s panda Oink oink Future headlines from the Multiverse Nov Trump How I Got Hillary Elected while Dismantling the Republican Party Awww That s the nicest thing anybody has said to me in a long while If ComicCon people ruled the world international conflicts would be resolved entirely by plastic light saber fights in bars On Pluto with its year orbit around the Sun birthdays are incompatible with human physiology Maybe I m floating in an atmospheric balloon in Saturn s atmosphere The urge to want some bit of information to be true often clouds our ability to assess why that information may be false Evidence that internet Cats are rapidly achieving cosmic consciousness soon to become our Overlords Ru1xRk feature share A word of our times DELUSION Being sure something is true in the face of all evidence that says it is not Odd that our measures of animal intelligence are often tests of what humans do best ra

In [3]:
#Finalmente, debemos convertir nuestra lista de lista en una única lista. Esto se puede conseguir con una función:

#Creamos la lista única:
flat_data = []

#Esta función indica que cada item (caracter) debe incluirse individualmente en la nueva lista (flat_data):
for sublist in data3:
    for item in sublist:
        flat_data.append(item)

#Comprobamos el resultado:
flat_data

['M',
 'o',
 'o',
 'n',
 ' ',
 's',
 ' ',
 's',
 'h',
 'a',
 'd',
 'o',
 'w',
 ' ',
 'l',
 'a',
 'n',
 'd',
 'f',
 'a',
 'l',
 'l',
 's',
 ' ',
 'O',
 'r',
 'e',
 'g',
 'o',
 'n',
 ' ',
 'c',
 'r',
 'o',
 's',
 's',
 'e',
 's',
 ' ',
 'U',
 'S',
 'A',
 ' ',
 'a',
 't',
 ' ',
 'm',
 'p',
 'h',
 ' ',
 'e',
 'x',
 'i',
 't',
 's',
 ' ',
 'S',
 'C',
 'a',
 'r',
 'o',
 'l',
 'i',
 'n',
 'a',
 ' ',
 'B',
 'e',
 'h',
 'o',
 'l',
 'd',
 ' ',
 'M',
 'u',
 'u',
 'u',
 'r',
 'i',
 'c',
 'a',
 ' ',
 's',
 ' ',
 'p',
 'a',
 'n',
 'd',
 'a',
 ' ',
 'O',
 'i',
 'n',
 'k',
 ' ',
 'o',
 'i',
 'n',
 'k',
 ' ',
 'F',
 'u',
 't',
 'u',
 'r',
 'e',
 ' ',
 'h',
 'e',
 'a',
 'd',
 'l',
 'i',
 'n',
 'e',
 's',
 ' ',
 'f',
 'r',
 'o',
 'm',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'M',
 'u',
 'l',
 't',
 'i',
 'v',
 'e',
 'r',
 's',
 'e',
 ' ',
 'N',
 'o',
 'v',
 ' ',
 'T',
 'r',
 'u',
 'm',
 'p',
 ' ',
 'H',
 'o',
 'w',
 ' ',
 'I',
 ' ',
 'G',
 'o',
 't',
 ' ',
 'H',
 'i',
 'l',
 'l',
 'a',
 'r',
 'y',
 ' ',
 'E',
 'l'

In [4]:
#Lo siguiente será limpiar esta nueva lista, de forma que solo conservemos las entradas únicas:
chars = sorted(list(set(flat_data)))

#Creamos dos diccionarios:
n_to_char = {n:char for n, char in enumerate(chars)} #Contiene los índices y su caracter asociado:
char_to_n = {char:n for n, char in enumerate(chars)} #Contiene los caracteres y su índice asociado:

#Comprobamos el resultado:
chars

[' ',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [5]:
#Para preprocesar datos en NLG, necesitamos crear dos listas:

#Una que contenga las frases, separadas por caracteres, y transformadas a sus índices correspondientes (char_to_n[char]):
X = []

#Otra que contenga el carácter siguiente del texto, según seq_length, convertida en índice (char_to_n[label]):
Y = []

#Guardamos la longitud de data3 en una variable:
length = len(data3)

#Fijamos el tamaño de las secuencias de caracteres. En este caso, 140 caracteres (como un tweet antigüo):
seq_length = 140

#Con esta función podemos completar las listas X e Y con los datos correspondientes:
for i in range(0, length-seq_length, 1):
    sequence = data3[i:i + seq_length]
    label = data3[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

In [6]:
#Redimensionamos X e Y para que sean datos de entrada aptos para el modelo:
X_ = np.reshape(X, (len(X), seq_length, 1))
X_ = X_ / float(len(chars))
Y_ = np_utils.to_categorical(Y)

In [15]:
#Una vez preparadas las listas, el siguiente paso es diseñar la arquitectura del modelo de generación de textos. Para crear
#el modelo, nos basaremos en modelos vistos en clase:

#Es un modelo secuencial
model = Sequential()
#Con una capa LSTM (habitual en NLG) de 100 neuronas y que acepta un input acorde a las dimensiones de los datos de entrada
model.add(LSTM(100, input_shape=(X_.shape[1], X_.shape[2]), return_sequences=True))
#Añadimos también regularización por dropout para evitar overfitting:
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
#Tras esto, una capa densa de salida, que acepta inputs acordes a los datos de salida (Y_)
model.add(Dense(Y_.shape[1], activation='softmax'))
#Compilamos el modelo, determinando el optimizador y la función de pérdidas:
model.compile(loss='categorical_crossentropy', optimizer='adam')

#En esta variable entrenaremos acorde a los parámetros establecidos:
history = model.fit(X_, Y_, validation_split=0.05, batch_size=512, epochs=3, shuffle=True)

Train on 236764 samples, validate on 12462 samples
Epoch 1/3
236764/236764 [==============================] - 1623s 7ms/step - loss: 3.0692 - val_loss: 2.9010
Epoch 2/3
236764/236764 [==============================] - 1695s 7ms/step - loss: 2.8676 - val_loss: 2.8204
Epoch 3/3
236764/236764 [==============================] - 1120s 5ms/step - loss: 2.8305 - val_loss: 2.7968


In [16]:
#Una vez diseñado el modelo, es necesaria una función capaz de generar el texto:
def generate_text(model, string_id):
    #Copia independiente de la string de entrada a la función:
    string_mapped = deepcopy(string_id)
    #Traducción a texto real de string_mapped:
    full_string = [n_to_char[value] for value in string_mapped]
    
    #Este bucle: 
    for i in range(400):
        #- Redimensiona los datos de entrada con cada nueva letra añadida (x):
        x = np.reshape(string_mapped,(1,len(string_mapped), 1))
        x = x / float(len(chars))

        #- Añade, tanto a índice (string_mapped) como a su texto asociado (full_string) el carácter con mayor
        #probabilidad de ser el carácter a insertar a continuación en el texto.
        pred_index = np.argmax(model.predict(x, verbose=2))
        full_string.append(n_to_char[pred_index])

        string_mapped.append(pred_index)
        string_mapped = string_mapped[1:len(string_mapped)]
    
    #Con esto, incluimos el texto real que precede a la generación del modelo, como referencia:
    text = ""
    for char in full_string:
        text = text + char
    return text

In [17]:
#Finalmente, ejecutamos el modelo (model) sobre la frase de entrada (X[0]):
new_text = generate_text(model, X[0]) 

In [121]:
#Observamos el resultado [para entrenamiento durante una sola época]:
new_text

'Moon s shadow landfalls Oregon crosses USA at mph exits SCarolina Behold Muuurica s panda Oink oink Future headlines from the Multiverse Novssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss'

In [18]:
#Observamos el resultado [para entrenamiento durante tres épocas]:
new_text

'Moon s shadow landfalls Oregon crosses USA at mph exits SCarolina Behold Muuurica s panda Oink oink Future headlines from the Multiverse Nov th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th th '

Hay una ligera diferencia de calidad en lo producido en una y en tres épocas de entrenamiento para el mismo modelo. Aun así, los resultados no son buenos, pues el modelo, aunque capaz de separar lo que considera palabras por espacios, no es capaz de generar frases coherentes. <br>

Una de las maneras de mejorar el resultado sería entrenar durante más épocas y con menor batch size, ya que por cuestiones de tiempo, computación y número de muestras del dataset, esto es todo lo que he podido hacer. También podríamos diseñar una arquitectura más compleja, añadiendo capas LSTM o incluyendo más neuronas.<br>

Otra posibilidad es utilizar alguno de los modelos pre-entrenados ya existentes en la red, como [ULMfit](https://github.com/jannenev/ulmfit-language-model). Con esto, entrenando durante pocas épocas, obtendríamos un resultado igual o superior al que obtendríamos entrenando el modelo actual durante muchas más épocas.<br>